In [1]:
import xml.etree.ElementTree as ET
from glob import glob
import subprocess
import os
import pandas as pd
from multiprocessing import Pool

In [2]:
program = "../../build_release/bin/ssipp"


In [3]:
timeout = 30

In [4]:
target_folder = {
    "../instances/singleagent-icaps2020/empty64x64/": "empty64x64.xml",
    "../instances/singleagent-icaps2020/warehouse/": "warehouse.xml",
    "../instances/singleagent-icaps2020/rooms/": "rooms.xml",
    "../instances/singleagent-icaps2020/den520d/": "den520d.xml"
                }




In [5]:
results = pd.DataFrame(columns = ["task", "lookahead", "learning algorithm", "solved", "solution length", "solution duration"])
lookaheads = ["10", "100", "1000"]
learnings = ["dijkstralearning", "plrtalearning"]
dynmode = ["0", "1"]
def parse_output(filepath):
    tree = ET.parse(filepath)
    root = tree.getroot()
    try:
        log = root.find("log")
        agent = log.find("agent")
        path = agent.find("path")
        return float(path.attrib["duration"]), len(path)
    except:
        return float("inf"), 0
    
def run_exp(config, task, lookahead, learning, dm):    
    tree = ET.parse(config)
    root = tree.getroot()
    alg = root.find("algorithm")
    learning_algorithm = alg.find("learningalgorithm")
    learning_algorithm.text = learning
    look = alg.find("fixedlookahead")
    look.text = lookahead
    dyn = ET.SubElement(alg, 'dynmode')
    dyn.text = dm
    outconfig = config.split("/")[-1]
    tree.write(outconfig)
    obs = task.replace(".xml", "_obs.xml")
    outfile = task.replace(".xml", "_log.xml")
    command = [program, task, config, outconfig, obs]
    try:
        subprocess.run(command, timeout = timeout)
    except:
        pass
    res = parse_output(outfile)
    return pd.Series(index = results.columns, data = (task, lookahead, learning, True, res[1], res[0]))

In [6]:
c = 0
exp_results = []
pool = Pool(4)
for cfg in target_folder:
    config = cfg + target_folder[cfg]
    tasks = glob(cfg+ "/*task.xml")
    for lookahead in lookaheads:
        for task in tasks:
            for learning in learnings:
                for dm in dynmode:
                    print(lookahead, task, learning, dynmode)
                    exp_results.append(pool.apply_async(run_exp, (config, task, lookahead, learning, dm)))

10 ../instances/singleagent-icaps2020/empty64x64/235_task.xml dijkstralearning ['0', '1']
10 ../instances/singleagent-icaps2020/empty64x64/235_task.xml dijkstralearning ['0', '1']
Warning! No 'prioritization' element found inside 'algorithm' section. Its value is set to 'fifo'.
Warning! No 'prioritization' element found inside 'algorithm' section. Its value is set to 'fifo'.
Warning! No 'timelimit' element found inside 'algorithm' section. Its value is set to 'infinity' (no limit).
Warning! No 'rescheduling' element found inside 'algorithm' section. Its value is to 'no'.
Warning! No 'planforturns' element found inside 'algorithm' section. Its value is set to 'false'.
Warning! No 'timelimit' element found inside 'algorithm' section. Its value is set to 'infinity' (no limit).
Warning! No 'rescheduling' element found inside 'algorithm' section. Its value is to 'no'.
Warning! No 'waitbeforemove' element found inside 'algorithm' section. Its value is set to '0'.
Warning! No 'planforturns' e

In [7]:
outres = []
for res in exp_results:
    outres.append(res.get())


In [ ]:
results = pd.concat(outres, axis = 1).T
results["solved"] = results["solution length"] != 0

In [ ]:
results.to_csv("more-results.csv")

In [ ]:
for lookahead in lookaheads:
    for learning in learnings:
        print(lookahead, learning)
        print(results.loc[(results.lookahead == lookahead) & (results["learning algorithm"] == learning), "solved"].sum())

,task,lookahead,learning algorithm,solved,solution length,solution duration
0,../instances/singleagent-icaps2020/empty64x64/...,10,dijkstralearning,True,40,60.84745
1,../instances/singleagent-icaps2020/empty64x64/...,10,dijkstralearning,True,40,60.84745


In [ ]:
!pwd